In [8]:
import sys
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text
from keras.layers import Dense, Activation, GRU
from keras.models import Sequential
import os

In [2]:
# Input size of the network, the entry text must have the same length
window_size = 100

In [3]:
def create_gru_model(number_chars):
    """
    Define the network
    :param
        numbers_chars .- Number chars using in the training process
    :return:
        model .- Model network defined
    """
    num_chars = len(number_chars)
    model = Sequential()
    # 1 Layer .- GRU layer 1 should be an GRU module with 200 hidden units
    model.add(GRU(200, input_shape=(window_size, num_chars), return_sequences=True))
    # 2 Layer .- GRU layer 2 should be an GRU module with 200 hidden units
    model.add(GRU(200))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(Dense(num_chars, activation='softmax'))
    return model

In [4]:
chars_to_indices, indices_to_chars = load_coded_dictionaries()

In [5]:
model = create_gru_model(chars_to_indices)
model.load_weights('model_weights/best_beiras_gru_textdata_weights.hdf5')

In [10]:
directory="./exports"
if not os.path.exists(directory):
    os.makedirs(directory)

https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

In [14]:
from keras import backend as K
from keras.models import model_from_config

K.set_learning_phase(0)  # all new operations will be in test mode from now on
# serialize the model and get its weights, for quick re-building
config = model.get_config()
weights = model.get_weights()

# re-build a model where the learning phase is now hard-coded to 0
new_model = Sequential.from_config(config)
new_model.set_weights(weights)

In [15]:
from tensorflow_serving.session_bundle import exporter
export_path = directory + "/beiras-rnn" # where to save the exported graph
export_version = 1.0

saver = tf.train.Saver(sharded=True)
model_exporter = exporter.Exporter(saver)
signature = exporter.classification_signature(input_tensor=new_model.input,
                                              scores_tensor=new_model.output)
model_exporter.init(sess.graph.as_graph_def(),
                    default_graph_signature=signature)
model_exporter.export(export_path, tf.constant(export_version), sess)

ModuleNotFoundError: No module named 'tensorflow_serving'